In [ ]:
import numpy as np
import cv2 as cv

In [ ]:
test_text = """AAAAAA
AAABBA
AAABBA
ABBAAA
ABBAAA
AAAAAA"""

test_data = test_text.split("\n")

In [ ]:
with open("input.txt") as f:
    data_text = f.readlines()
input_data = [s.replace("\n", "") for s in data_text]

In [ ]:
def get_arr(text: list) -> np.ndarray:
    return np.array([[j for j in i] for i in text])

In [ ]:
def check_idx(idx: tuple, indexes: list) -> int:
    dps = []
    for i in [-1, 1]:
        dps.append((idx[0] + i, idx[1]))
        dps.append((idx[0], idx[1] + i))
    res = 0
    for dp in dps:
        if dp not in indexes:
            res += 1
    return res

def search_arr(regions_arr: np.ndarray, idx: tuple, indexes: list, region_val: int):
    indexes.remove(idx)
    regions_arr[idx] = region_val
    dps = []
    for i in [-1, 1]:
        dps.append((idx[0] + i, idx[1]))
        dps.append((idx[0], idx[1] + i))
    for dp in dps:
        if dp in indexes:
            search_arr(regions_arr, dp, indexes, region_val)

def split_regions(mask: np.ndarray) -> list:
    regions_arr = mask.astype(int)
    region_val = 2
    while np.any(regions_arr == 1):
        ids = np.where(regions_arr == 1)
        indexes = [(int(x), int(y)) for x, y in zip(ids[0], ids[1])]
        search_arr(regions_arr,indexes[0], indexes, region_val)
        region_val += 1
    return regions_arr

def get_regions_cnt(mask: np.ndarray) -> int:
    regions = split_regions(mask)
    regions_uniques = np.unique(regions)
    regions_uniques = regions_uniques[regions_uniques != 0]
    return len(regions_uniques)

### Part one

In [ ]:
data_arr = get_arr(input_data)
uniques = np.unique(data_arr)
total_cost = 0
for element in uniques:
    mask = data_arr == element
    regions = split_regions(mask)
    regions_uniques = np.unique(regions)
    regions_uniques = regions_uniques[regions_uniques != 0]
    for unique in regions_uniques:
        region = regions == unique
        ids = np.where(regions == unique)
        indexes = [(int(x), int(y)) for x, y in zip(ids[0], ids[1])]
        perimeter = 0
        for idx in indexes:
            perimeter += check_idx(idx, indexes)
        cost = perimeter * len(indexes)
        total_cost += cost
print(total_cost)

### Part two

In [ ]:
def extract_boundaries(region: np.ndarray):
    extended = np.zeros((region.shape[0] + 2, region.shape[1] + 2))
    extended[1:-1, 1:-1] = region[:, :]
    
    kernel = np.ones((3, 3), np.uint8) 
    borders = cv.dilate(extended, kernel)
    borders[1:-1, 1:-1] -= region[:, :]
    return borders


In [ ]:
def get_region_edges_stripe_wise(region: np.ndarray) -> list:
    border = extract_boundaries(region)
    border_regions = split_regions(border == 1)
    regions_uniques = np.unique(border_regions)
    regions_uniques = regions_uniques[regions_uniques != 0]
    extended = np.zeros((region.shape[0] + 2, region.shape[1] + 2))
    extended[1:-1, 1:-1] = region[:, :]
    edges = 0
    for j in range(1, extended.shape[1] - 1):
        is_in_border_left = False
        is_in_border_right = False
        for i in range(1, extended.shape[0] - 1):
            left_empty = extended[i, j - 1] == 0 
            right_empty = extended[i, j + 1] == 0 
            if extended[i, j] == 1 and left_empty:
                if not is_in_border_left:
                    edges += 1                    
                    is_in_border_left = True    
            else:
                is_in_border_left = False
            
            if extended[i, j] == 1 and right_empty:
                if not is_in_border_right:
                    edges += 1                    
                    is_in_border_right = True    
            else:
                is_in_border_right = False

    for i in range(1, extended.shape[0] - 1):
        is_in_border_top = False
        is_in_border_down = False
        for j in range(1, extended.shape[1] - 1):
            top_empty = extended[i - 1, j] == 0 
            down_empty = extended[i + 1, j] == 0 
            if extended[i, j] == 1 and down_empty:
                if not is_in_border_down:
                    edges += 1                    
                    is_in_border_down = True    
            else:
                is_in_border_down = False
            
            if extended[i, j] == 1 and top_empty:
                if not is_in_border_top:
                    edges += 1                    
                    is_in_border_top = True    
            else:
                is_in_border_top = False
            
    return edges

In [ ]:
data_arr = get_arr(input_data)
uniques = np.unique(data_arr)
total_cost = 0
for element in uniques:
    print(element)
    mask = data_arr == element
    regions = split_regions(mask)
    regions_uniques = np.unique(regions)
    regions_uniques = regions_uniques[regions_uniques != 0]
    for unique in regions_uniques:
        region = regions == unique
        perimeter = get_region_edges_stripe_wise(region)
        indexes = np.where(region == True)
        indexes = [(int(x), int(y)) for x, y in zip(indexes[0], indexes[1])]
        cost = perimeter * len(indexes)
        total_cost += cost
print(total_cost)